## Setup and Imports

In [1]:
!pip install -q datasets

In [2]:
import torch
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.notebook import tqdm

## Model Loading

In [3]:
def load_model_and_tokenizer(model_name):
    """Load model and tokenizer with multi-GPU support"""
    # Check GPU availability
    if not torch.cuda.is_available():
        raise RuntimeError("No GPU available")
    
    n_gpus = torch.cuda.device_count()
    print(f"Found {n_gpus} GPUs")
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
    
    # Load model with optimal settings for T4 GPUs
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,  # Use float16 for T4 GPUs
        device_map="auto",         # Automatically handle multi-GPU
        max_memory={i: "12GiB" for i in range(n_gpus)},  # T4 has 16GB but leave some headroom
    )
    model.eval()
    
    # Get device (first GPU)
    device = torch.device("cuda:0")
    
    return model, tokenizer, device

## Data Loading

In [16]:
def load_mmlu_data(subjects=None, languages=['ru', 'en']):
    """Load MMLU data for specified subjects and languages"""
    if subjects is None:
        subjects = ['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics', 'clinical_knowledge', 'college_biology', 'college_chemistry', 'college_computer_science', 'college_mathematics', 'college_medicine', 'college_physics', 'computer_security', 'conceptual_physics', 'econometrics', 'electrical_engineering', 'elementary_mathematics', 'formal_logic', 'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_computer_science', 'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics', 'high_school_macroeconomics', 'high_school_mathematics', 'high_school_microeconomics', 'high_school_physics', 'high_school_psychology', 'high_school_statistics', 'high_school_us_history', 'high_school_world_history', 'human_aging', 'human_sexuality', 'international_law', 'jurisprudence', 'logical_fallacies', 'machine_learning', 'management', 'marketing', 'medical_genetics', 'miscellaneous', 'moral_disputes', 'moral_scenarios', 'nutrition', 'philosophy', 'prehistory', 'professional_accounting', 'professional_law', 'professional_medicine', 'professional_psychology', 'public_relations', 'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions']
    
    dfs = []
    for subject in subjects:
        for lang in languages:
            try:
                if lang == 'ru':
                    dataset = load_dataset("NLPCoreTeam/mmlu_ru", subject, split="test")
                else:
                    dataset = load_dataset("cais/mmlu", subject, split="test")
                df = dataset.to_pandas()
                df['subject'] = subject
                df['language'] = lang
                dfs.append(df)
            except Exception as e:
                print(f"Error loading {subject} in {lang}: {e}")
    
    return pd.concat(dfs, ignore_index=True)

## Evaluation Functions

In [17]:
def format_prompt(row):
    """Format a single question into a prompt based on language"""
    lang = row['language']
    
    if lang == 'ru':
        prompt = f"""Ответьте на вопрос, выбрав правильный вариант (A, B, C или D).

Вопрос: {row['question_ru'] if 'question_ru' in row else row['question']}

Варианты ответа:
"""
        choices = row['choices_ru'] if 'choices_ru' in row else row['choices']
        for idx, choice in enumerate(choices):
            prompt += f"{chr(65 + idx)}. {choice}\n"
        prompt += "\nОтвет (укажите только букву A, B, C или D):"
        
    else:  # English
        prompt = f"""Answer the question by selecting the correct option (A, B, C, or D).

Question: {row['question']}

Options:
"""
        for idx, choice in enumerate(row['choices']):
            prompt += f"{chr(65 + idx)}. {choice}\n"
        prompt += "\nAnswer (provide only the letter A, B, C, or D):"
    
    return prompt

def evaluate_model(model, tokenizer, df, device, debug_samples= -1, batch_size=4):
    """Evaluate model on the dataset with debugging information and batch processing"""
    results = []

    # Process data in batches
    for i in tqdm(range(0, len(df), batch_size)):
        batch_df = df.iloc[i:i + batch_size]
        # Create a list of prompts for the current batch
        prompts = [format_prompt(row) for _, row in batch_df.iterrows()]

        # Tokenize all prompts in the batch
        inputs = tokenizer(prompts, 
                           return_tensors="pt", 
                           padding=True, 
                           truncation=True).to(device)

        # Generate for the entire batch
        with torch.no_grad():
            try:
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=10,
                    do_sample=False,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                    use_cache=True
                )
            except Exception as e:
                print("Error during model generation:", e)
                continue

        # Iterate over each sample in the batch
        for idx, (prompt, (_, row)) in enumerate(zip(prompts, batch_df.iterrows())):
            # Decode the model's output for the current sample
            response = tokenizer.decode(outputs[idx], skip_special_tokens=True)
            # Remove the prompt from the beginning of the response
            generated = response[len(prompt):].strip()

            # Get the correct answer and convert if needed
            correct_answer = row['answer']
            if str(correct_answer).isdigit():
                correct_answer = chr(65 + int(correct_answer))  # Convert number to letter
            correct_answer = str(correct_answer).upper()

            # Extract the first capital letter as the prediction
            pred = next((c for c in generated if c.upper() in 'ABCD'), 'X')
            pred = pred.upper()

            # Print debug information for the first few samples
            if idx < debug_samples:
                print(f"\nDebug Sample {idx + 1} ({row['language']}):")
                print(f"Question: {row['question_ru'] if 'question_ru' in row else row['question']}")
                print(f"Full Response: {generated}")
                print(f"Extracted Prediction: {pred}")
                print(f"Correct Answer: {correct_answer}")
                print(f"Choices: {row['choices_ru'] if 'choices_ru' in row else row['choices']}")

            results.append({
                'subject': row['subject'],
                'language': row['language'],
                'question': row['question_ru'] if 'question_ru' in row else row['question'],
                'correct_answer': correct_answer,
                'predicted_answer': pred,
                'full_response': generated,
                'correct': pred == correct_answer
            })

    return pd.DataFrame(results)


## Running Evaluation

In [6]:
# Run these commands in separate cells:

# 1. Load model
model_name = "Qwen/Qwen2.5-3B-Instruct"  # e.g., "Qwen/Qwen2.5-1.5B-Instruct"
model, tokenizer, device = load_model_and_tokenizer(model_name)

Found 2 GPUs


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [18]:
# 2. Load data
eval_df = load_mmlu_data(languages=['en'])
eval_df_ru = load_mmlu_data(languages=['ru'])

test-00000-of-00001.parquet:   0%|          | 0.00/20.1k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/5.28k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/135 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/14 [00:00<?, ? examples/s]

...

0000.parquet:   0%|          | 0.00/9.02k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/41.9k [00:00<?, ?B/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/19 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/171 [00:00<?, ? examples/s]

In [ ]:
# 3. Run evaluation
results_df = evaluate_model(model, tokenizer, eval_df, device)
results_df_ru = evaluate_model(model, tokenizer, eval_df_ru, device)

  0%|          | 0/3511 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [ ]:
# 4. Calculate and display results for english
accuracy = results_df['correct'].mean()
subject_accuracy = results_df.groupby('subject')['correct'].mean()

print(f"Overall accuracy: {accuracy:.2%}")
print("\nAccuracy by subject:")
print(subject_accuracy)

In [ ]:
# 4. Calculate and display results for russian
accuracy = results_df_ru['correct'].mean()
subject_accuracy = results_df_ru.groupby('subject')['correct'].mean()

print(f"Overall accuracy: {accuracy:.2%}")
print("\nAccuracy by subject:")
print(subject_accuracy)

In [ ]:
# 5. Save results
results_df.to_csv(f"mmlu_results_{model_name.replace('/', '_')}.csv", index=False)